In [1]:
import fcalc

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score

Reading data

In [2]:
all_data = pd.read_csv('heart.csv')
object_cols = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'slp', 'caa', 'thall']
for col in object_cols:
    all_data[col] = all_data[col].astype('object')

Binarizing data

In [3]:
def max_qcut(series):
    num_cut = 2
    while(True):
        try:
            pd.qcut(series, num_cut)
        except ValueError:
            return num_cut-1
        else:
            num_cut+=1

In [6]:
qcut = 6

X, y = all_data.drop(columns = 'output').copy(), all_data['output'].copy()

qcut_cols = X.select_dtypes(['int64', 'float64']).columns

for column in qcut_cols:
    X[column] = pd.qcut(X[column], min(qcut, max_qcut(X[column])))

In [5]:
X = pd.get_dummies(X)

In [6]:
X.shape

(303, 52)

Assesing performance

In [7]:
def cross_val_score(X, y, n_splits, scoring):
    skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    scores = np.zeros((len(scoring), skf.get_n_splits()))

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        bin_cls = fcalc.classifier.BinarizedBinaryClassifier(X_train.values, y_train.values)
        bin_cls.predict(X_test.values)

        # we always classify non prediction as false prediction
        tmp_predict = np.copy(bin_cls.predictions)
        tmp_predict[tmp_predict == -1] = np.abs(y_test.values[tmp_predict == -1] - 1)

        for j, score in enumerate(scoring):
            scores[j, i] = score(y_test.values, tmp_predict)
    
    return scores.mean(1).round(3)


In [8]:
cross_val_score(X, y, 5, [accuracy_score, f1_score])

array([0.845, 0.858])